**A brief primer on APL**

The entire language of APL consists of 83ish symbols like +, ↑, and ⍣. Many symbols are functions which behave differently when used *monadically*, i.e. with only a single right-hand argument, and *dyadically*, i.e. with both right- and left-hand arguments.

For example, "⌈" means "ceiling" when used monadically:

In [17]:
⌈4.5

5

And means "maximum" when used dyadically:

In [18]:
1⌈5

5

Some symbols are operators, meaning instead of taking value arguments, some or all of their arguments are functions. For example, ¨ (each) maps its function over an array:

In [19]:
⌈¨2.3 5.1 6.6 4

3 6 7 4

Finally, many functions exhibit rank polymorphism, meaning they can take arrays of values any place they could take values. For example, you don't really need ¨ in the previous example:

In [21]:
⌈2.3 5.1 6.6 4

3 6 7 4

That should be enough to get started! If you want to learn more, I recommend the [Learning APL book](https://xpqz.github.io/learnapl).

**A note**

Throughout this tutorial, I truncate results using "take" (↑) to make the output more readable. Just know if you see something like `3↑...`, it means you're only viewing the first 3 elements in the result and you should imagine the "..." that follows it.

First, a bit of preamble...

In [4]:
]box on
]rows on
⎕PW←120

Was ON

Was ON

**Parsing the input**

Let's start by parsing the input data. This is the magic incantation for reading a file line-by-line. Don't worry about how it works, but if you want to know more, [here are the details](https://xpqz.github.io/learnapl/io.html?highlight=nl#reading-text-files-nget). You can see that we get an array of strings, where some of the strings are empty.

In [85]:
data←⊃⎕NGET'input.txt'1
20↑ data

┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬┬────┬────┬────┬────┬────┬────┐
│3427│3273│5615│5943│3125│4245│4194│3243│4283│1790│5355│4239│5541││3850│5294│5393│2490│3231│2743│
└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴┴────┴────┴────┴────┴────┴────┘

Now let's split it up into groups separated by those empty elements. To do that, we'll us partition (⊆), which takes our data on the right, and a binary vector on the left. Everywhere where the binary vector is 0, we'll get a split. For example:

In [17]:
1 1 0 1 0 1 1⊆⍳7

┌───┬─┬───┐
│1 2│4│6 7│
└───┴─┴───┘

To know where to split the data, we need 1s where there's a non-empty string, and 0s where there's not. Tally (≢) gives us the length of a vector (strings are just character vectors), and direction (×) turns positive numbers into 1s and leaves 0s alone. (It also turns negative numbers into -1s, but that's not important right now.)

So all we need to do is map (¨) tally (≢) over the data, then apply × to the whole thing:

In [86]:
50↑ ×≢¨data

1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1

That gives us the binary vector we need for the left argument to partition (⊆). 

We could write it like this:

In [87]:
2↑ (×≢¨data)⊆data

┌──────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────┐
│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│
││3427│3273│5615│5943│3125│4245│4194│3243│4283│1790│5355│4239│5541│││3850│5294│5393│2490│3231│2743│2016│2418│2304│5673│3035│5292│2112│2630│2305││
│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│
└──────────────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────────────────────┘

But anytime you need a value on both sides of an operator, you can instead use special construct called a fork. A fork follows this form:

```
(f x) h (g x) = (f h g) x
```

In my case, I have `(×≢¨data)⊆data = ((×≢¨)⊆⊢)data`. The ⊢ is just the identity function, because I don't want to do anything to `data` in the right-hand argument. If I leave the data out, you can see the structure of the fork:

In [29]:
((×≢¨)⊆⊢)

┌──┼─┐
┌┴┐ ⊆ ⊢
× ¨
┌─┘
≢

In [88]:
2↑ ((×≢¨)⊆⊢)data

┌──────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────┐
│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐│
││3427│3273│5615│5943│3125│4245│4194│3243│4283│1790│5355│4239│5541│││3850│5294│5393│2490│3231│2743│2016│2418│2304│5673│3035│5292│2112│2630│2305││
│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│└────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘│
└──────────────────────────────────────────────────────────────────┴────────────────────────────────────────────────────────────────────────────┘

Finally, to turn the strings into numbers, I use execute (⍎). Notice I have to map (¨) twice because my values are two arrays deep:

In [89]:
parsed←⍎¨¨((×≢¨)⊆⊢)data
2↑ parsed

┌────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────┐
│3427 3273 5615 5943 3125 4245 4194 3243 4283 1790 5355 4239 5541│3850 5294 5393 2490 3231 2743 2016 2418 2304 5673 3035 5292 2112 2630 2305│
└────────────────────────────────────────────────────────────────┴──────────────────────────────────────────────────────────────────────────┘

**Part one solution**

The rest is easy! For part one, we need to find the subarray with the greatest sum. Let's first sum the subarrays. We can add two things with +, so to sum the array, we just need to reduce (/) using +.

In [90]:
20↑ +/¨parsed

54273 50786 47383 48088 42497 48625 48083 58597 53996 42727 51398 41338 54225 46142 49899 47586 50030 40939 57297 48647

Finally, we take the biggest one using maximum (dyadic ⌈):

In [48]:
⌈/+/¨parsed

68775

And that's the answer to part one!

**Part two solution**

For part two, we need the sum of the greatest 3 subarrays. That means we need to sort the data in descending order. I'm not going to go into the details of the sorting incantation, but [you can read more about it here](https://xpqz.github.io/learnapl/manip.html?highlight=sort#grade-up-down).

In [91]:
sort←(⊂∘⍒⌷⊢)
20↑ sort+/¨parsed

68775 67045 66765 65924 64498 64481 63207 63101 63049 62125 61387 61230 61016 60528 59931 59704 59348 58597 58355 58222

Then we just take the first 3 and sum-reduce:

In [92]:
3↑sort+/¨parsed
+/3↑sort+/¨parsed

68775 67045 66765

202585

And that's our answer for part two!